<a href="https://colab.research.google.com/github/astrid12345/recyclo/blob/mk_test_train/src/train_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a bare-bones file that demonstrates downloading a dataset from google drive and using it to train a YOLOv8 model.

Consider this a placeholder only - if you're training a model for real, please overhaul this file however you deem necessary!

In [3]:
!pip install -U ultralytics gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [1]:
google_drive_file_id = '1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg'  # <-- From sharing link in google drive, eg https://drive.google.com/file/d/1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg/view?usp=sharing
google_drive_file_name = '20250623_merged_yolo_reshuffled.zip'

In [2]:
import zipfile
from pathlib import Path
import gdown
from ultralytics import YOLO
import yaml

project_root = Path('/content')
dataset_root_name = 'yolo_dataset'
output_path = project_root / google_drive_file_name
extract_dir = project_root / dataset_root_name
dataset_root_path = Path('/content') / dataset_root_name
yaml_path = dataset_root_path / 'data.yaml'

# Download from Google Drive and extract
gdown.download(id=google_drive_file_id, output=str(output_path), quiet=False)
print(f"Dataset downloaded from google drive: {output_path}\n")

Downloading...
From (original): https://drive.google.com/uc?id=1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg
From (redirected): https://drive.google.com/uc?id=1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg&confirm=t&uuid=fbe32ecb-6847-4d4b-97fe-7148095a707e
To: /content/20250623_merged_yolo_reshuffled.zip
100%|██████████| 3.98G/3.98G [00:52<00:00, 76.3MB/s]

Dataset downloaded from google drive: /content/20250623_merged_yolo_reshuffled.zip



In [4]:
print("Extracting...\n")
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Dataset extracted to: {extract_dir}\n", )

# Update path in data.yaml to work with where we've unzipped it
print("Updating data.yaml path...\n")
with yaml_path.open('r') as file:
    data = yaml.safe_load(file)

data['path'] = str(dataset_root_path)

with yaml_path.open('w') as file:
    yaml.dump(data, file)

print(f"Updated path in data.yaml to: {data['path']}")

Extracting...

Dataset extracted to: /content/yolo_dataset

Updating data.yaml path...

Updated path in data.yaml to: /content/yolo_dataset


In [13]:
# Sanity check that it's possible to train a model with the dataset
dataset_yaml = Path('/content/yolo_dataset/data.yaml')
model = YOLO('yolo11n.pt')
results = model.train(data=str(dataset_yaml), epochs=10, imgsz=640)

print(f"Training complete. Results saved to: {model.trainer.save_dir}")

Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /content/yolo_dataset/labels/train.cache... 3472 images, 1 backgrounds, 0 corrupt: 100%|██████████| 3472/3472 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1017.0±968.9 MB/s, size: 917.6 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 434 images, 0 backgrounds, 0 corrupt: 100%|██████████| 434/434 [00:00<?, ?it/s]


...

      10/10      3.37G      1.081      0.937      1.133         21        640: 100%|██████████| 217/217 [02:22<00:00,  1.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:14<00:00,  1.00s/it]

                   all        434        838      0.826      0.521      0.588      0.406



10 epochs completed in 0.466 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 5.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:23<00:00,  1.70s/it]


                   all        434        838      0.824      0.521      0.588      0.407
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train4
Training complete. Results saved to: runs/detect/train4
